# Part 2: Clustering for  Single Family Rental in Georgia State

Single-Family Rental(SFR) is a label assigned to standalone rental properties. SFRs are sometimes called single-family residential properties.
The aim of this project is to provide insight on how we can use clustering to identify outliers in the housing market, characterize and identify different kinds of anomalies, identify oportunities for investment in each neighborhood and environment. 
It provides data backed insight for an SFR Investor who wish to invest in a place with long term growth by highlighting different areas with increased gentrification correlated with property prices.



Aim: Develop a system of visualisation of the different clusters in GA SFR to guide a potential investor in making optimum investment choice.

https://www.kaggle.com/code/eisgandar/car-prices-predict-with-ensemble-methods
    https://www.kaggle.com/code/eisgandar/customer-segmentation-with-k-means/notebook

In [ ]:
# Import the required modules
import pandas as pd
import numpy as np

import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Import the clean dataset for Ga_listing
gadf = pd.read_csv("json_rest_sep26.csv")
# we subset to data that concerns single family residence
#gadf = gadf[gadf["Type__Detached"]==1]

gadf = gadf.iloc[:, 1:]
lst1 = gadf[gadf["price"]> 150000]  

# Filter by price to retain only like single family rentals
gadf = lst1[lst1["price"]<1000000]
gadf.head()

In [ ]:
# Sort values by price and drop duplicates
gadf = gadf.sort_values(by= "price")
gadf.drop_duplicates(inplace=True)
gadf


In [ ]:
# Subset to where there are less than 10 beds nad 9 baths, and less than 20000 sq ft of lot size
gadf = gadf[gadf.beds<10]
gadf = gadf[gadf.baths_full<10]
gadf=gadf[gadf.lot_size<20000]
gadf.price =gadf.price.apply(lambda x: x/100000)
gadf.square_footage =gadf.square_footage.apply(lambda x: x/1000)
gadf.year_built =gadf.year_built.apply(lambda x: x/10)
gadf.aland_sqmi =gadf.aland_sqmi.apply(lambda x: x/10)
#.drop("4yrs_max")


In [ ]:
gadf.aland_sqmi.value_counts()

In [ ]:
# Restrict special feautes to maximum of 3 and replace square footage 0 by the mean.

gadf.special_features =np.where(gadf.special_features>3, 3, gadf.special_features)
gadf.square_footage =np.where(gadf.square_footage==0, gadf.square_footage.mean(), gadf.square_footage)

In [ ]:
# Visualise the scatter plots of each element against price
for ele in gadf.iloc[:, 3:32].columns :
    print(ele.upper())
    plt.scatter(gadf[ele], gadf.price)
    #plt.bar(gadf[ele], gadf.price)
    plt.show()

# 2.2 Principal Component Analysis

PCA is used for:

(1) Dimensionality Reduction
If we have a high-dimensional dataset (the one we're working with has 44 dimenions), it is computationally more expensive to create predictions. Moreover with a greater amount of features, there's a greater risk of overfitting.

(2) Visualization (in EDA). This is what we're going to do here. As it is we cannot visualise all features, (we can only visualize 3 dimensions)
PCA allows us to visualize our dataset along two or three most important principal components


# 2.2.1 Normalisation of the Data

Some of the obtained values are very large and we need to scaled to avoid dominance by these extreme values. The principal components may vary if the columns involved are not somewhat homogenous.

In [ ]:
# Drop unneccesary columns and create new useful features.

gadf3 = gadf   #.drop([ "city_new","NationalRank", "county_new"], axis =1) # drop repeated columns

gadf3.head()

In [ ]:
# Rescale the data and prepare it for PCA

gadf3["2020pop"] = gadf3["2020pop"]/10000
gadf3["2019pop"] = gadf3["2019pop"]/10000
gadf3["2018pop"] = gadf3["2018pop"]/10000
gadf3["2017pop"] = gadf3["2017pop"]/10000

gadf3["pop_growth_2020"] = (100*(gadf3["2020pop"]- gadf3["2019pop"])/(gadf3["2019pop"]))
gadf3["pop_growth_2019"] = (100*(gadf3["2019pop"]- gadf3["2018pop"])/(gadf3["2018pop"]))
gadf3["pop_growth_2018"] = (100*(gadf3["2018pop"]- gadf3["2017pop"])/(gadf3["2017pop"]))


In [ ]:
# Create new feature on population growth

gadf3["4yrs_min_pop_growth"] = list(map(lambda w,x,y,z: min(w,x,y,z),gadf3["pop_growth_2022"], gadf3["pop_growth_2020"],gadf3["pop_growth_2019"],gadf3["pop_growth_2018"]))
     
     
gadf3["avg_growth"] = list(map(lambda w,x,y,z: (w+x+y+z)/4,gadf3["pop_growth_2022"], gadf3["pop_growth_2020"],gadf3["pop_growth_2019"],gadf3["pop_growth_2018"]))
     
gadf3["long"] = gadf3["Long"]
gadf3["lat"] = gadf3["Lat"]

gadf3 = gadf3.drop(["Long", "Lat"], axis= 1)

# Scale columns with very large input values with the log function.
new_lst = ['year_built','Percapita','Median_household_income', 'Median_family_income','Numberof_households2015',  'Avg.Income/H/hold']
for ele in new_lst:
    gadf3["{}".format(ele)] = list(map(lambda x: 0 if x==0 else np.log(abs(float(x))+ 1.01), (gadf3["{}".format(ele)])))
    




In [ ]:
# create new feature from the old ones and drop the old ones
gadf3["county"] =gadf3["County"]
gadf3["Zip_Code"] =gadf3["Zip Code"]# Some of the predictors have high corelation. To avoid this multicollinearity, we need to run PCA to retain the most relevant predictors.
#gadf3.drop(["County", "Zip Code"],axis=1)
gadf3 = gadf3.drop(["unit_count","County","Zip Code"], axis=1)


In [ ]:
# Determine the value counts of each feature
for ele in gadf3.columns:
    print(gadf3[ele].value_counts())

In [ ]:
# Visualise how each predictor affects the Price based on Location. We do this using seaborn package

fig = plt.figure(figsize=(12, 9))
lstc =gadf3.iloc[:, 2:31].columns
for ele in lstc:
    print("Which area has the highest", ele.upper())
    fig = plt.figure(figsize=(12, 10))
    print(ele.upper())
    sns.scatterplot(gadf3.iloc[:,1],gadf3.iloc[:,0], s= list(gadf3["price"].apply(lambda x: 10*(1))), c=gadf3[ele])
    plt.show()

In [ ]:
# Check for missing data
gadf3.isna().sum()

In [ ]:
# check for possible anomaly in the 2D locations with respect to each of the predictors using pyplot in Matplotlib
import matplotlib.pyplot as plt
for ele in gadf3.iloc[:,2:31].columns:
    print("Which area has the highest", ele.upper())
    fig = plt.figure(figsize=(13, 10))
    plt.scatter(gadf3.iloc[:,1], gadf3.iloc[:,0],s=6, c= gadf3["{}".format(ele)])
    plt.show()

# Principal Component Analysis

PCA is used for dimensionality reduction / feature selection when the feature space contains too many irrelevant or redundant features. The aim is to find the intrinsic dimensionality of the data. It allows to project the data from the original 42-dimensional space into a lower dimensional space. Subsequently, we can project into a 3-dimensional space and plot the data and the clusters in this new space.

In [ ]:
# Import relevant libraries 

pd.options.display.float_format = '{:.2f}'.format

np. set_printoptions(suppress=True)
import seaborn as sns 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import plotly.express as px
from math import sin, cos, pi

In [ ]:
# plot the heatmap of the features
gadf3.corr().style.background_gradient(cmap = "copper")

In [ ]:
breaks = [3,12,16,21,25,28,35,38] # Deternine the number of columns to run PCA with.
pca=PCA(n_components=2)
exp_ratio = []
# We will not use the longitude, latitude and price in the PCA to make sure that our result is blind to locations and price.
data =gadf3[["longitude", "latitude", "price"]] # We assume that longitude, latiude are independent predictors.
pca.set_params(n_components=2)

for i in range(6):
    principal_components_ = pca.fit_transform(gadf3.iloc[:,breaks[i]:breaks[i+1]])
    

    data1 =gadf3[["longitude", "latitude"]]
    # Visualize data across the linear components
     # Create a new dataframe for the PCA values
 
    total_var = sum(pca.explained_variance_ratio_)*100
    data1["PCA_1"] =   list(principal_components_[:,0]) # Add the first pricipal component to the data1
    
    exp_ratio.append(pca.explained_variance_ratio_[0]) 
    
    data["PCA_"+"{}".format(i)] = list(principal_components_[:,0])
    if pca.explained_variance_ratio_[1]>0.25:
        data["PCA_2"+"{}".format(i)] = list(principal_components_[:,1])
        exp_ratio.append(pca.explained_variance_ratio_[1]) 
    #fig = plt.figure(figsize=(15, 10))
    fig = px.scatter_3d(
        np.array(data1), x=0, y=1, z=2, color=gadf3['price'],
        title=f'Total Explained Variance: {total_var:.2f}%,  PCA_1:  {100*pca.explained_variance_ratio_[0]:.2f}%, PCA_2: {100*pca.explained_variance_ratio_[1]:.2f}% ',
        labels={"Longitude", "Latitude", "PCA_"}

    )
    fig.show()
    

In [ ]:
# Check the data
data.head()

In [ ]:
# Create a new datframe
pca_df = data
data

In [ ]:
# Visualise any three combinations as it relates to price
from itertools import combinations
  
# Get all combinations of [1, 2, 3]
# and length 2
comb = combinations(list(data.iloc[:,3:].columns), 3)
  
# Print the obtained combinations
for i in list(comb):
    
    data1 =data[["{}".format(i[0]), "{}".format(i[1]),"{}".format(i[2])]]
    #print(data1.head())
    fig = px.scatter_3d(
        np.array(data1), x=0, y=1, z=2, color=0,
        title=f'Variables:     {i[0]}, : {i[1]}, {i[2]}',
        
    )
    fig.show()


In [ ]:
# Visulise the heatmap of the new data
pca_df.corr().style.background_gradient(cmap = "copper") #get the correlations of each features in dataset. From the corrmatrix, we can see that there is low colinearity among the features.
# That shows that the features are linearly independent

The heatmap shows that multi collinearity is low

### Scaling the Data With Standard Scaler

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
scaled = pd.DataFrame(scaler.fit_transform(data.iloc[:,3:]))
scaled


# KMeans Clustering

Whereas the PCA seeks to represent all 𝑛 features as linear combinations of a small number of eigenvectors, and does it to minimize the mean-squared error, in contrast, the K-means seeks to represent all 𝑛 data vectors via small number of cluster centroids. That is, it seeks to represent them as linear combinations of a small number of cluster centroid vectors where linear combination weights must be all zero except for the single 1.
This is also done to minimize the mean-squared error.
Thus both K-means and PCA minimize the same objective function. The difference is that K-means has additional "categorical" constraint.


Clustering helps in understanding the natural grouping in a dataset. Their purpose is to make sense to partition the data into some group of logical groupings.



In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline

In [ ]:
scaled

In [ ]:
#plot Elbow method
# K-means starts with allocating cluster centers randomly and then looks for "better" solutions. 
#One thing about this algorithm is that I have to give the number of clusters beforehand, so I'll be using the WCSS (elbow method) to come up with a more accurate idea.
#plt.plot(range(2, 15), wcss)

#plt.xlabel('Number of clusters')
#plt.ylabel('WCSS') 
#plt.show()
# From the graph we probaly need only 7 clusters. That is, after 12, clusters, the inertia stabilizes

# Kmeans With Random init

In [ ]:
# Visualize the cluster
X=scaled
x =pca_df.longitude
y= pca_df.latitude
z = pca_df.price
fig = plt.figure(figsize=(15, 10))

kmeans = KMeans(n_clusters=7 ,init = "random",random_state=0).fit(X)
plt.scatter(x, y, c=kmeans.labels_, s=3)
plt.title("Kmeans K=7 with random init")
plt.show()
df_new = gadf3
df_new["Cluster"] = kmeans.labels_


# Kmeans with init k-means++

In [ ]:
# Try a different initilization with k-means++
x =pca_df.longitude
y= pca_df.latitude
z = pca_df.price
fig = plt.figure(figsize=(15, 10))

kmeans = KMeans(n_clusters=7 ,init = "k-means++",random_state=0).fit(X)
plt.scatter(x, y, c=kmeans.labels_, s=3)
plt.title("K-means++, K=7")
plt.show()
df_new = gadf3
df_new["Cluster+"] = kmeans.labels_


# MiniBatchKmeans

In [ ]:
# Try MiniBatch
from sklearn.cluster import MiniBatchKMeans
kmeans2 = MiniBatchKMeans(n_clusters=7, random_state=0, batch_size=21)
kmeans2 = kmeans2.partial_fit(X)
#kmeans2 = kmeans2.partial_fit(X[:,21:])
kmeans2.cluster_centers_
mb_label = kmeans2.predict(X)
fig = plt.figure(figsize=(15, 10))
df_new["mb_Cluster"] = mb_label
plt.scatter(x, y, c=mb_label, s=3)
plt.title("MiniBatchKmeans with K=7")
plt.show()

# Agglomerative Hierarchical Clustering

In [ ]:
# Try Agglomerative Hierachical Clustering

from sklearn.cluster import AgglomerativeClustering
import numpy as np
 
X = scaled
 
# here we need to mention the number of clusters
# otherwise the result will be a single cluster
# containing all the data
for linkage in ("ward", "average", "complete", "single"):
    clustering2 = AgglomerativeClustering(linkage=linkage,affinity = 'euclidean', n_clusters=7)
    clustering2.fit(X)
    df_new[linkage] = clustering2.fit_predict(X)
    fig = plt.figure(figsize=(11, 8))
    plt.scatter(x, y, c=clustering2.labels_, s=3)
    plt.title("Agglomerative Hierarchical Clustering with linkage {} K=7".format(linkage))
    plt.show()

In [ ]:
# return the hierarchical clustering using Ward Method encoded as a linkage matrix (ndarray)
# Plot the dendogram  - a method that tries to minimize the variance within each cluster.
dendrogram = sch.dendrogram(sch.linkage(X, method  = "ward"))
plt.title('Dendrogram')
plt.xlabel('Properties')
plt.ylabel('Euclidean distances')
plt.show()

# DBSCAN

In [ ]:
# Try DBSCAN
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs

#centers =np.array(df_new.groupby("Cluster").mean())#[["longitude", "latitude"]])

X, labels_true = make_blobs(
    n_samples=len(pca_df), centers=7, cluster_std=0.1, random_state=0
)
X=scaled



#X = StandardScaler().fit_transform(X)
db = DBSCAN(eps=0.1, min_samples=100).fit(X)


core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
df_new["db_Cluster"] = labels
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = 7
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
print(
    "Adjusted Mutual Information: %0.3f"
    % metrics.adjusted_mutual_info_score(labels_true, labels)
)
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X, labels))
fig = plt.figure(figsize=(11, 8))
plt.scatter(x, y, c=labels, s=3)
plt.title("DBSCAN Clustering with linkage  K=7")
plt.show()


# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]


In [ ]:
df_new.iloc[:,25:]

In [ ]:
df_new.iloc[:,:].groupby("Cluster").count() # These are the clusters produced by Kmeans random

In [ ]:
# df_new.iloc[:,18:].groupby("Cluster+").mean() # These are the clusters produced by Kmeans k-means++

In [ ]:
df_new.iloc[:,:].groupby("mb_Cluster").count() # These are the clusters produced by Kmeans

In [ ]:
df_new.iloc[:,:].groupby("ward").count() # These are the clusters produced by Agglomerative Hierarchical Clustering

In [ ]:
#df_new.iloc[:,:].groupby("Cluster3").count() # AGGlomerative CLustering complete

In [ ]:
df_new.iloc[:,:].groupby("db_Cluster").count() # These are the clusters produced by MiniBatchKmeans

K-Means++ is a smart centroid initialization technique that handles issues of convergence that exists in the traditional Kmeans and the rest of the algorithm is the same as that of K-Means.


In [ ]:
df_new.iloc[:,25:].groupby("Cluster+").mean() # These are the clusters produced by Kmeans random

In [ ]:
# Visualize k-means++ Clusters 
for ele  in range (7):
    df_new1 = df_new.groupby("Cluster+").get_group(ele)
    
    x =df_new1.longitude
    y= df_new1.latitude
    z = df_new1.price
    pcd=df_new1.iloc[:,:3]
    fig = plt.figure(figsize=(15, 10))
    print("Cluster_", ele)
    
    plt.scatter(x, y, c=df_new1.groc_6km, s=3)
    plt.title("Cluster_{}".format(ele))
    plt.show()

In [ ]:
df_new.groupby("Cluster").count()

In [ ]:
df_new.iloc[:,20:].groupby("Cluster").mean()

# Validation of the Models

In [ ]:
# Validate the models using different scores

from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(
            data,
            estimator[-1].labels_,
            metric="euclidean",
            sample_size=100,
        )
    ]

    # Show the results
    formatter_result = (
        "{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}"
    )
    print(formatter_result.format(*results))

# Run the Benchmark

In [ ]:
data = data.iloc[:,3:]

In [ ]:
# Validate the models using different scores
n_digits=7

print(82 * "_")

print("init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette")

kmeans = KMeans(init="k-means++", n_clusters=n_digits, n_init=40, random_state=0)

bench_k_means(kmeans=kmeans, name="k-means++", data=data, labels=labels)
kmeans = KMeans(init="random", n_clusters=n_digits, n_init=40, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=data, labels=labels)

pca = PCA(n_components=7).fit(data)
kmeans = KMeans(init=pca.components_, n_clusters=7, n_init=40)
bench_k_means(kmeans=kmeans, name="PCA-based", data=data, labels=labels)

print(82 * "_")

# Analysis of Subclusters

In [ ]:
# Get the groups and check the various characteristics of the groups
for i in range(len(df_new.groupby("Cluster+")["Cluster+"].unique())) :   
    Y = df_new.groupby("Cluster+").get_group(i)
    plist = [] # The plist scores the averge price in a cluster
    low = 0.667*gadf["price"].min()+ 0.333*gadf["price"].max()
    medium = 0.33*gadf["price"].min()+ 0.667*gadf["price"].max()

    for ele in sorted(Y["price"])[::-1]:

        if ele <low:
            plist.append(1)
        if ele >=low and ele <medium:
            plist.append(2)
        if ele >= medium:
            plist.append(3)
    Y["plist"] = plist
    #dic = []
    #dic.append([1, low, Y["price"].max()])
    fig = plt.figure(figsize=(11, 8))
    print("Cluster_",i)
    sns.scatterplot(Y["longitude"],Y["latitude"],hue=Y["plist"],palette="pastel")
    plt.show()
    print("Summary of Group {}".format(i), "\n", Y.describe())


In [ ]:
# Get the groups and check the various characteristics of the groups
for i in range(len(df_new.groupby("Cluster+")["Cluster+"].unique())) :   
    Y = df_new.groupby("Cluster+").get_group(i)
    plist = [] # The plist scores the averge price in a cluster
    low = 0.667*gadf["Crime_Rating"].min()+ 0.333*gadf["Crime_Rating"].max()
    medium = 0.33*gadf["Crime_Rating"].min()+ 0.667*gadf["Crime_Rating"].max()

    for ele in sorted(Y["Crime_Rating"])[::-1]:

        if ele <low:
            plist.append(1)
        if ele >=low and ele <medium:
            plist.append(2)
        if ele >= medium:
            plist.append(3)
    Y["plist"] = plist
    #dic = []
    #dic.append([1, low, Y["price"].max()])
    fig = plt.figure(figsize=(11, 8))
    print("Cluster_",i)
    sns.scatterplot(Y["longitude"],Y["latitude"],hue=Y["plist"],palette="pastel")
    plt.show()
    print("Summary of Group {}".format(i), "\n", Y.describe())


In [ ]:
# Get the groups and check the various characteristics of the groups
for i in range(len(df_new.groupby("Cluster+")["Cluster+"].unique())) :   
    Y = df_new.groupby("Cluster+").get_group(i)
    plist = [] # The plist scores the averge price in a cluster
    low = 0.667*gadf["avg_5yr_sfrgrowth"].min()+ 0.333*gadf["avg_5yr_sfrgrowth"].max()
    medium = 0.33*gadf["avg_5yr_sfrgrowth"].min()+ 0.667*gadf["avg_5yr_sfrgrowth"].max()

    for ele in sorted(Y["avg_5yr_sfrgrowth"])[::-1]:

        if ele <low:
            plist.append(1)
        if ele >=low and ele <medium:
            plist.append(2)
        if ele >= medium:
            plist.append(3)
    Y["plist"] = plist
    #dic = []
    #dic.append([1, low, Y["price"].max()])
    fig = plt.figure(figsize=(11, 8))
    print("Cluster_",i)
    sns.scatterplot(Y["longitude"],Y["latitude"],hue=Y["plist"],palette="pastel")
    plt.show()
    print("Summary of Group {}".format(i), "\n", Y.describe())


### Further Analsis of the SubClusters

In [ ]:
from IPython.display import Image
Image("picture.png")

From the Clusters, we could infer that Clusters 0,2,5 are relatively low priced while Clusters 1,6 are relatively high priced.
The Crime rating of Clusters 6 and 1 are the lowest, cluster 3 is moderate while cluster 0 is very high. The growth rate of Cluster 0 and 3 are the highests.
Also Clusters 0 and 3 have maintened high Sfr growth in the last 5 years while Cluster 1 and 6 have show some decline. 
This is unconnected to Covid-19 that might have moved some higher income earners to relocate to lower income residential areas.
Cluster 4 seems to have more amenities while Cluster 0 has fewer amenities. Clusters 0 has maintained a high population growth in the last 4 years while Cluster 4 has not been so lucky.

Overall, Cluster 3 seems to be the best investment option when cost is not considered.
On the other hand a low budget investor may consider Cluster 1 with the hope that amenities will eventually improve and Crime_Rating reduce




In [ ]:
for ele  in range (7):
    df_new2 = df_new.groupby("Cluster+").get_group(ele)
    
    x =df_new2.longitude
    y= df_new2.latitude
    z = df_new2.price
    #pcd=df_new1.iloc[:,:3]

    #mask=point_cloud.price>=poi.price.min()


    fig = plt.figure(figsize=(15, 10))
    print("Cluster_", ele)
    #kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
    plt.scatter(x, y, c=df_new2.groc_6km, s=3)
    plt.title("Cluster{}".format(ele))
    plt.show()
    #df_new = gadf3
    #df_new["Cluster"] = kmeans.labels_

# 3D Representations of the Clusters

In [ ]:
# import the modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN
import scipy.cluster.hierarchy as sch
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Import the Data Sets
#gadf_new = gadf3# pd.read_csv("json_rest_newest.csv")
# we subset to data that concerns single family residence
point_cloud = gadf3   #[(gadf_new["Type__Condo"]==1) | (gadf_new["Type__Townhouse"]==1) | (gadf_new["Type__Detached"]==1)]
#point_cloud["County"] = point_cloud["county_new"]
#point_cloud =point_cloud.iloc[:,1:]
point_cloud.iloc[:,20:38]

In [ ]:
# Scale the columns to make them homogenous
#point_cloud["square_footage"] = point_cloud["square_footage"].apply(lambda x: x/1000)
#point_cloud["year_built"] = point_cloud["year_built"].apply(lambda x:x/100)
#point_cloud["growth"] = point_cloud["growth"].apply(lambda x: 100*x)
#point_cloud["price"] = point_cloud["price"].apply(lambda x: x/1000)

In [ ]:
x =point_cloud.longitude
y= point_cloud.latitude
z = point_cloud.price
pcd=point_cloud.iloc[:,:3]
mask=point_cloud.price>0
spatial_query=pcd[point_cloud.price>np.mean(point_cloud.price)]


#This is the cluster that has only 18 features.
X= point_cloud.iloc[:,:37] #np.column_stack((x[mask], y[mask], z[mask], illuminance[mask], nb_of_returns[mask], intensity[mask]))
fig = plt.figure(figsize=(15, 10))

kmeans = KMeans(n_clusters=7, init = 'k-means++', random_state=0).fit(X)
#plt.scatter(x[mask], y[mask], c=kmeans.labels_, s=3)

plt.show()
df_new["Cluster4"] = kmeans.labels_
#df_new.groupby("Cluster4").count()

In [ ]:
x =point_cloud.longitude
y= point_cloud.latitude
z = point_cloud.price
pcd=point_cloud.iloc[:,:3]
mask=point_cloud.price>0
spatial_query=pcd[point_cloud.price>np.mean(point_cloud.price)]


#This is the cluster that has only 18 features.
#X= point_cloud.iloc[:,:27] #np.column_stack((x[mask], y[mask], z[mask], illuminance[mask], nb_of_returns[mask], intensity[mask]))
fig = plt.figure(figsize=(15, 10))

kmeans = KMeans(n_clusters=7, init = "k-means++", random_state=0).fit(X)
plt.scatter(x[mask], y[mask], c=list(kmeans.labels_), s=3)

plt.show()
kmeans.labels_

In [ ]:
#plotting the result in  2D
cols = point_cloud.iloc[:,24:29].columns

for ele in cols:
    fig = plt.figure(figsize=(10, 8))
    print(ele.upper())
    plt.scatter(point_cloud.longitude[mask],point_cloud.latitude[mask], c=point_cloud["{}".format(ele)][mask], s=10)
    plt.show()

In [ ]:
# We need at least 7 cluster because of the context of this study. We run Kmean to determine the optimum number of clusters. We tabulate the inertia and select the number K that has the lowest innertia.
X=point_cloud.iloc[:,:27]#np.column_stack((x[mask], y[mask], z[mask])) # We want to minimize the inertia.
wcss = [] 
for i in range(3, 10):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    print(i,", ", np.min(wcss))
    #fig = plt.figure(figsize=(15, 10))
    #plt.scatter(x[mask], y[mask], c=kmeans.labels_, s=5)
    #plt.show()

In [ ]:
#plot Elbow method
# K-means starts with allocating cluster centers randomly and then looks for "better" solutions. One thing about this algorithm is that I have to give the number of clusters beforehand, so I'll be using the WCSS (elbow method) to come up with a more accurate idea.
plt.plot(range(3, 10), wcss)
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') 
plt.show()
# From the graph we probaly need only 18 clusters. That is, after 18, clusters, the inertia stabilizes

In [ ]:
#This is the cluster that has only 18 features.
X= point_cloud.iloc[:,:27] #np.column_stack((x[mask], y[mask], z[mask], illuminance[mask], nb_of_returns[mask], intensity[mask]))
fig = plt.figure(figsize=(12, 8))

kmeans = KMeans(n_clusters=7, random_state=0).fit(X)
plt.scatter(x[mask], y[mask], c=kmeans.labels_, s=3)

plt.show()

In [ ]:
point_cloud["labels"] = list(kmeans.labels_)

In [ ]:
#point_cloud.groupby("labels").get_group(9)

In [ ]:
point_cloud.iloc[:,39:]

In [ ]:
#data3.iloc[:,:]

In [ ]:
# Visualise the crime rate in 3D
from mpl_toolkits.mplot3d import Axes3D
colors = ["black","blue","green","purple","red","#FF0800", "yellow", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]
fig = plt.figure(figsize=(12, 8))
ax=Axes3D(fig)
labels =list(range(7))
data3 =df_new
ax.set_xlabel('$Longitude$', fontsize=20, rotation=150)
ax.set_ylabel('$Latitude$', fontsize=20, rotation=150)
ax.set_zlabel('$Crime-Rating$', fontsize=30, rotation=60)
for ele in labels:
    for i in range(len(data3)):
        if data3["Cluster"].iloc[i,] == ele:
            ax.scatter(data3.iloc[i,1],data3.iloc[i,0],data3.iloc[i,27],color=colors[ele])

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#colors = ["orange", "blue","green",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]
fig = plt.figure(figsize=(12, 8))
ax=Axes3D(fig)
labels =list(range(7))
data3 =df_new
ax.set_xlabel('$Longitude$', fontsize=20, rotation=150)
ax.set_ylabel('$Latitude$', fontsize=20, rotation=150)
ax.set_zlabel('$rest- 6km?$', fontsize=30, rotation=60)
for ele in labels:
    for i in range(len(data3)):
        if data3["Cluster"].iloc[i,] == ele:
            ax.scatter(data3.iloc[i,1],data3.iloc[i,0],data3.iloc[i,26],color=colors[ele])

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12, 8))
ax=Axes3D(fig)
labels =list(range(7))
#data3 =point_cloud
ax.set_xlabel('$Longitude$', fontsize=20, rotation=150)
ax.set_ylabel('$Latitude$', fontsize=20, rotation=150)
ax.set_zlabel('$groc - 6km?$', fontsize=30, rotation=60)
for i in range(len(point_cloud)):
    for ele in labels:
        if point_cloud["Cluster"].iloc[i,] == ele:
            ax.scatter(point_cloud.iloc[i,1],point_cloud.iloc[i,0],point_cloud.iloc[i,25],color=colors[ele])

plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
#colors = ["red","blue","black",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]
fig = plt.figure(figsize=(12, 8))
ax=Axes3D(fig)
labels =list(range(7))
data3 =df_new
ax.set_xlabel('$Longitude$', fontsize=20, rotation=150)
ax.set_ylabel('$Latitude$', fontsize=20, rotation=150)
ax.set_zlabel('$Clusters$', fontsize=30, rotation=60)
for ele in labels:
    for i in range(len(data3)):
        if data3["Cluster"].iloc[i,] == ele:
            ax.scatter(data3.iloc[i,1],data3.iloc[i,0],data3.iloc[i,39],color=colors[ele])

plt.show()

In [ ]:
df_new.groupby("Cluster+").get_group(3).iloc[:, 33:]["county"].unique()

# Data Visualisation with Geopandas

In [ ]:
# Visualise the data on geopandas.
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.colors as colors


import folium
df_new2 = df_new
cmap1=colors.ListedColormap(["orange", "blue","black",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"])
df_new2["Cluster"] = df_new["Cluster"]
# define the geometry
geometry = [Point(xyz) for xyz in zip(df_new2['longitude'], df_new2['latitude'])]
gdf = GeoDataFrame(df_new2, geometry=geometry) 

m = gpd.GeoDataFrame(geometry=geometry, crs="epsg:4326").explore(name="Georgia", height=500, width=900,tiles = 'openstreetmap', zoom_start=7,min_zoom=6,max_zoom=10, cmap=cmap1)
# plot the points, outcomes as different colors
#folium.TileLayer('Stamen Terrain').add_to(m2)
#folium.TileLayer('Stamen Toner').add_to(m2)
#folium.TileLayer('Stamen Water Color').add_to(m2)
#folium.TileLayer('cartodbpositron').add_to(m2)
#folium.TileLayer('cartodbdark_matter').add_to(m2)
m = gdf.explore(m =m, cmap=colors.ListedColormap(["orange", "blue","black",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]), name="points")
# add layer control so layers can be switched on / off
folium.LayerControl().add_to(m)

m


In [ ]:
#Determine the fastest growing area by population
    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

m = gdf.explore(m =m, cmap=colors.ListedColormap(["orange", "blue","black",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]), name="points")
# add layer control so layers can be switched on / off
folium.LayerControl().add_to(m)

gdf['Hemisphere'] = list(map(lambda x: str(x),gdf["Cluster+"]))
cluster = gdf.explore(column='Hemisphere',tiles = "openstreetmap", cmap=colors.ListedColormap(["orange", "blue","black",'#5CD925','#D94325',"purple","#800000","#FF0000", "#800080", "#FF00FF", "#008000","#00FF00", "#808000","#FFFF00","#006400", "#000080","#0000ff","#6495ed", "#0000FF","#008080","#00FFFF"]))
#folium.TileLayer('Stamen Terrain').add_to(cluster)
folium.LayerControl().add_to(cluster)
cluster                                                 

In [ ]:
#Determine the center of the distribution 
        
    
#long =-84.3880
#lat = 33.7490

#gdf['Hemisphere'] = list(map(lambda x,y: 'Norte'if (x-long)**2+(y-lat)**2 <0.02  else  "Yellow",gdf["longitude"],gdf["latitude"]))
#gdf.explore(column='Hemisphere', cmap=colors.ListedColormap(['#5CD925','#D94325',"yellow"]))

#gdf['Hemisphere'] = list(map(lambda x,y: 'Norte'if (x-long)**2+(y-lat)**2 <0.02  else 'Sur',gdf["longitude"],gdf["latitude"]))
#gdf.explore(column='Cluster', cmap=colors.ListedColormap(['green',"red","blue","black",'#D94325','#5CD925','#D94325']))

In [ ]:
#df["Median_family_income"].mean()

In [ ]:
#Determine the fastest growing area by population
    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
#gdf['Hemisphere'] = list(map(lambda x: 'Norte'if x<2.4   else  "Yellow",gdf["square_footage"]))
#gdf.explore(column='Hemisphere', cmap=colors.ListedColormap(['#5CD925','#D94325',"yellow"]))

In [ ]:
#Determine the fastest growing area by population
    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
#gdf['Hemisphere'] = list(map(lambda x: 'Norte'if x >7  else  "Yellow",gdf["Median_family_income"]))
#gdf.explore(column='Hemisphere', cmap=colors.ListedColormap(['#5CD925',"yellow"]))

In [ ]:
#Determine the neighbourhoods with high prices 
    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
gdf['Hemisphere'] = list(map(lambda x: 'Norte'if x>9   else  "Yellow",gdf["price"]))
gdf.explore(column='Hemisphere', cmap=colors.ListedColormap(['yellow','#D94325']))

In [ ]:
#Determine the neighbourhoods with new buildings
    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
gdf['Hemisphere'] = list(map(lambda x: 'Orange'if x>0 and x<2   else  "Yellow",gdf["year_built"]))
#gdf['Price'] = list(map(lambda x: 'Norte'if x>1000000   else  "Yellow",gdf["price"]))
gdf.explore('Hemisphere', cmap=colors.ListedColormap(['#FFA500',"green"]))

In [ ]:
#Determine areas with very low average growth
        
    

    
#gdf = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))
gdf['Hemisphere'] = list(map(lambda x,y: 'Norte'if y<1.1   else  "Black",gdf["4yrs_min_pop_growth"], gdf["avg_growth"]))
gdf.explore(column='Hemisphere', cmap=colors.ListedColormap(['#5CD925','black',"yellow"]))